In [44]:
import jet
import numpy as np

In [45]:
# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
H = jet.Tensor(["i", "k"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

In [46]:
# Defining CNOT gate
# We can view it as a rank-4 tensor of size 2 by 2 by 2 by 2
# Two of the legs of this tensor represent inputs and 
# the other two legs represent outputs
CNOT = jet.Tensor(["k", "j", "l", "m"], [2, 2, 2, 2])
CNOT.set_value((0, 0, 0, 0), 1) # |00> -> |00>
CNOT.set_value((0, 1, 0, 1), 1) # |01> -> |01>
CNOT.set_value((1, 0, 1, 1), 1) # |10> -> |11>
CNOT.set_value((1, 1, 1, 0), 1) # |11> -> |10>

In [47]:
# Defining Pauli-Z Gate
pauli_z_data = [1, 0, 0, -1]
size = [2,2]
indices = ["l","n"]
Z = jet.Tensor(indices, size, pauli_z_data)

In [48]:
# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
new_H = jet.Tensor(["r", "t"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

In [49]:
# Defining CNOT gate
# We can view it as a rank-4 tensor of size 2 by 2 by 2 by 2
# Two of the legs of this tensor represent inputs and 
# the other two legs represent outputs
new_CNOT = jet.Tensor(["n", "m", "r", "s"], [2, 2, 2, 2])
new_CNOT.set_value((0, 0, 0, 0), 1) # |00> -> |00>
new_CNOT.set_value((0, 1, 0, 1), 1) # |01> -> |01>
new_CNOT.set_value((1, 0, 1, 1), 1) # |10> -> |11>
new_CNOT.set_value((1, 1, 1, 0), 1) # |11> -> |10>

In [54]:
# Explicitly define tensors with indices to mimic quantum circuit
# Contract network to get resulting tensorproduct of paulis
tn = jet.TensorNetwork()
tn.add_tensor(H)
tn.add_tensor(CNOT)
tn.add_tensor(Z)
tn.add_tensor(new_H)
tn.add_tensor(new_CNOT)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result)



[[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


The above matrix is equal to $X \otimes I$, which is what we expect

In [25]:
# Let us try with a simple non-Clifford gate, by introducing the T gate
angle = (np.pi/4)*1j
angle_p = -(np.pi/4) * 1j
T = jet.Tensor(["a","b"], [2,2], [1,0,0,np.exp(angle)])
T_p = jet.Tensor(["c","d"], [2,2],[1,0,0,np.exp(angle_p)])

In [26]:
# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
H = jet.Tensor(["e", "f"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])
H_p = jet.Tensor(["f", "g"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

In [27]:
# Defining Pauli-Z Gate
pauli_z_data = [1, 0, 0, -1]
size = [2,2]
indices = ["b","c"]
Z = jet.Tensor(indices, size, pauli_z_data)

In [29]:
tn = jet.TensorNetwork()
tn.add_tensor(T);
tn.add_tensor(Z);
tn.add_tensor(T_p);
tn.add_tensor(H);
tn.add_tensor(H_p);
result = tn.contract()
print(result)

Size = 4
Indices = {a  d}
Data = {(1,0)  (0,0)  (0,0)  (-1,0)}


In [30]:
pauli_x_data = [0,1,1,0]
size = [2,2]
indices = ["a","b"]
X = jet.Tensor(indices, size, pauli_x_data)

identity = jet.Tensor(["c","d"], [2,2], [1,0,0,1])

In [34]:
tn = jet.TensorNetwork()
tn.add_tensor(X)
tn.add_tensor(identity)


Size = 4
Indices = {a  b}
Data = {(0,0)  (1,0)  (1,0)  (0,0)}
